<a href="https://colab.research.google.com/github/tutsilianna/Gift_Recommendation_System/blob/main/Gift_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Аннотация

Список литературы:
1. https://education.yandex.ru/handbook/ml/article/intro-recsys
2. Анализ текстовых данных https://habr.com/ru/companies/otus/articles/774498/
---
https://neurohive.io/ru/tutorial/sozdaem-rekomendatelnuju-sistemu-s-ispolzovaniem-biblioteki-fastai/

https://habr.com/ru/companies/skillfactory/articles/585182/   

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B

https://vc.ru/u/1389654-machine-learning/592278-5-luchshih-generatorov-sinteticheskih-dannyh-na-python-i-kak-ih-ispolzovat-kogda-vam-ne-hvataet-dannyh

### Есть три варианта реализации:


1.  Content-Based Filtering (Фильтрация на основе контента)

    **Принцип:** Анализирует характеристики подарков и предпочтения пользователя (пол, возраст, интересы, хобби).
    
    **Данные:** Описания подарков, профили пользователей.
    
    **Алгоритмы:**TF-IDF, cosine similarity.
    
    **Пример:** Если пользователь увлекается фотографией, система порекомендует фотоаппарат, объектив или аксессуары для фотосъемки.

2. Collaborative Filtering (Коллаборативная фильтрация):

    **Принцип:** Анализирует историю покупок и оценок пользователей, чтобы найти похожих пользователей и рекомендовать подарки, которые понравились им.

    **Данные:** История покупок, рейтинги, отзывы.

    **Алгоритмы:** K-Nearest Neighbors, Matrix Factorization.

    **Пример:** Если пользователь А и пользователь Б имеют похожую историю покупок, и пользователь А купил книгу, которая понравилась пользователю Б, система порекомендует эту книгу другим пользователям с похожими предпочтениями.



3. Hybrid Approach (Гибридный подход):

    **Принцип:** Комбинирует content-based и collaborative filtering для повышения точности рекомендаций.

    **Пример:** Система использует content-based filtering, чтобы найти подарки, соответствующие интересам пользователя, а затем использует collaborative filtering, чтобы ранжировать эти подарки на основе предпочтений похожих пользователей.



* Дополнительные факторы:

    **Бюджет пользователя:** Система должна учитывать бюджет пользователя при рекомендации подарков.

    **Повод:** Подарки на день рождения, Новый год или свадьбу будут отличаться.

    **Социальная информация:** Информация о друзьях и семье пользователя в социальных сетях может помочь в подборе подарков.

**Входные параметры:** (опционально) пол, возраст, интересы/хобби - строковое описание увлечений человека.

**Данные для обучения:** даные о пользователе (входные параметры) + какие подарки он выбрал. Эти данные будут синтетическими

## Подключение репозитория

In [1]:
! git clone https://github.com/tutsilianna/Gift_Recommendation_System

Cloning into 'Gift_Recommendation_System'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 60 (delta 20), reused 24 (delta 5), pack-reused 0
Receiving objects: 100% (60/60), 15.89 MiB | 12.81 MiB/s, done.
Resolving deltas: 100% (20/20), done.


## Импорт необходимых библиотек

In [3]:
import re
import nltk
import json
import random
import warnings
import pandas as pd
import numpy as np
from nltk import stem
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download("stopwords")
warnings.filterwarnings("ignore")

mystem = Mystem()
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Чтение данных и предобработка

In [4]:
data = pd.read_csv('/content/Gift_Recommendation_System/data/raw/ozon_data_.csv')
data.head(2)

,название,описание,код товара,оценка,количество отзывов,цена,категория,подкатегория,подподкатегория,ссылка
0,Проектор для смартфона 2-го поколения своими р...,Проектор для смартфона (проектор для смартфона...,Артикул: 1320591182,3.3,13,808,Электроника,Телевизоры и видеотехника,Проекторы,https://www.ozon.ru/product/proektor-dlya-smar...
1,"Проектор 90411567, белый",3D увеличитель экрана телефона – это универсал...,Артикул: 619939085,4.4,205,300,Электроника,Телевизоры и видеотехника,Проекторы,https://www.ozon.ru/product/proektor-90411567-...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22591 entries, 0 to 22590
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   название            22591 non-null  object 
 1   описание            16366 non-null  object 
 2   код товара          21916 non-null  object 
 3   оценка              22591 non-null  float64
 4   количество отзывов  22591 non-null  int64  
 5   цена                22591 non-null  int64  
 6   категория           22591 non-null  object 
 7   подкатегория        22591 non-null  object 
 8   подподкатегория     20793 non-null  object 
 9   ссылка              22591 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 1.7+ MB


## **1. Подготовка данных**



### **Предобработка данных:** обработка пропущенных значений, удаление дубликатов, преобразование столбца "код товара";    


In [6]:
data['подподкатегория'] = data['подподкатегория'].fillna('')
data['код товара'] = data['код товара'].fillna('Артикул: 0')
data['описание'] = data['описание'].fillna('')

In [7]:
data.drop_duplicates(inplace=True)

In [8]:
# удаление слова из "Код товара", удаление строк,
# где нет атрибута и преобразование столбца к численному типу

data['код товара'] = data['код товара'].str.split().str.get(-1)
data = data.loc[data['код товара'].str.isdigit()]
data['код товара'] = data['код товара'].astype(np.int64)

In [9]:
data.drop_duplicates(subset=['код товара'], keep='first', inplace=True)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18427 entries, 0 to 22589
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   название            18427 non-null  object 
 1   описание            18427 non-null  object 
 2   код товара          18427 non-null  int64  
 3   оценка              18427 non-null  float64
 4   количество отзывов  18427 non-null  int64  
 5   цена                18427 non-null  int64  
 6   категория           18427 non-null  object 
 7   подкатегория        18427 non-null  object 
 8   подподкатегория     18427 non-null  object 
 9   ссылка              18427 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 1.5+ MB


### **Составление набора данных в виде анкет:** с помощью GPT4 была получена переменная `test`, в которой каждому интересу из переменной `interests_list` была сопоставлена подподкатегория из переменной `categories_list`


In [11]:
# Список интересов
interests_list = ["спорт", "музыка", "кино", "книги", "игры", "кулинария",
                  "путешествия", "искусство", "технологии", "мода", "красота",
                  "шоппинг", "автомобили", "рыбалка", "охота", "финансы",
                  "образование", "здоровье", "саморазвитие", "домашние животные",
                  "садоводство", "фотография", "танцы", "йога", "волонтерство",
                  "рисование", "живопись", "скульптура",
                  "театр", "писательство", "поэзия", "рукоделие",
                  "дизайн", "марки", "монеты", "комиксы", "антиквариат",
                  "игрушки", "видеоигры", "компьютерные игры", "мобильные игры",
                  "настольные игры", "азартные игры"]

categories_list = [category for category in set(data['подподкатегория'].values) if category != '']
categories = [f'{x[0]}/{x[1]}/{x[2]}' for x in data.groupby(['категория', 'подкатегория'])['подподкатегория'].value_counts().index]
print(f'Количество интересов: {len(interests_list)},\nКоличество категорий вариант №1: {len(categories_list)},\nКоличество категорий вариант №2: {len(categories)}')

Количество интересов: 43,
Количество категорий вариант №1: 277,
Количество категорий вариант №2: 331


` interests_categories = { "интерес": ['подподкатегория', ..],  ... }`

In [12]:
interests_categories = {
    "спорт":             ["Красота, здоровье и спорт", "Cпортивные игры"],

    "музыка":            ["Искусство и культура", "Музыкальные инструменты", "Уход за музыкальными инструментами",
                          "Гитары и аксессуары", "Струнные инструменты", "Ударные и перкуссия", "Клавишные инструменты"],

    "кино":              ["Искусство и культура", "Кино", 'Проекторы', 'Экраны и проекторы'],

    "книги":             ["Книги", "Художественная литература", "Нехудожественная литература", "Проза", "Поэзия",
                          "Сентиментальная проза", "Религиозные книги", "Биографии и истории успеха", "Юридическая литература",
                          "Саморазвитие", "Компьютерные технологии", "Итальянский язык", "Наборы", "Другие языки", "Энциклопедии и справочники",
                          "Медицина", "История", "Фантастика", "Публицистика", "Английский язык", "Эзотерика и спиритизм", "Французский язык",
                          "Путешествия. Путеводители", "Учебная литература", "Пьесы и драматургия", "Товары для рукоделия",
                          "Научно-популярная литература", "Фольклор", "Молодежная и подростковая литература", "Немецкий язык",
                          "Античная литература", "Научная литература", "Детям и родителям", "Инвестиции и недвижимость",
                          "Финансы", "Манга", 'Детективы', "Фэнтези", "Приключения", "Ужасы и триллеры"],

    "игры":              ["Игры", "Игровые приставки", "Игры для Nintendo Switch", "Игры для Xbox Series", "Игры для Xbox 360",
                          "Игры для PlayStation Portable/Vita", "Игры для Xbox One", "Игры для всей семьи (Family)",
                          "Игры для Nintendo Wii/3DS", "Игры для PlayStation 4", "Игры для PlayStation 5",
                          "Игры для PlayStation 2/3", "Настольные игры", "Дополнения к настольным играм",
                          "Ходилки",  "Робототехника", 'Боевики', 'Геймпады и джойстики', 'Гонки (Racing)', 'Детективы', 'Игровые адаптеры',
                          "Очки виртуальной реальности для смартфонов",  "Видеоигры", "Компьютерные игры",
                          "Мобильные игры", "Настольные игры", "Азартные игры", "Приключения (Adventure)", "Игровые консоли",
                          "Аркады (Arcade)", "Ролевые игры (RPG & MMORPG)", "Стратегии (Strategy)", "Дополнения", "Лото",
                          "Сюжетно-ролевые игрушки", "Симуляторы (Simulation)", "Модели и фигурки",
                          "Ужасы (Horror)", "Драки (Fighting)", "Шутеры и экшн (Shooter & Action)", "Квесты и логика (Quest & Logic)",
                          "Интерактивные и электронные игрушки", "Военная техника и оружие", "Фигурки и аксессуары", "Гонки",
                          "Фэнтези", "Приключения", "Ужасы и триллеры","Спортивные игры (Sports)"],

    "кулинария":         ["Кулинария", "Кулинарные книги", "Домоводство и хобби", "Тарелки и блюда", "Предметы сервировки", "Стекло и всё для бара"],

    "путешествия":       ["Путешествия. Путеводители", "Сувениры и подарки"],

    "искусство":         ["Искусство и культура", "Материалы и инструменты для ювелиров", "Пигменты",
                          "Рисование жидким акрилом (Флюид-арт)", "Наборы для лепки", "Мольберты и этюдники",
                          "Штампы и трафареты", "Картины из пайеток", "Картины из эпоксидной смолы",
                          "Авторская посуда", "Средства для очищения кистей", "Топперы для торта",
                          "Вазы, кашпо", "Выжигание и поделки из дерева", "Создание украшений",
                          "Пластилин", "Глина для лепки", "Реставрация и золочение", "Инструменты",
                          "Авторские игрушки", "Бумага для черчения и рисования",
                          "Наборы для квиллинга", "Шелкография", "Духовые инструменты",
                          "Скетчбуки", "Краски", "Уголь, сангина, сепия и соус", "Маркеры",
                          "Наборы для росписи", "Подставки", "Картриджи для 3D ручек", "Шитье",
                          "Поделки из бумаги", "Веники для бань", "Изготовление ножей", "Ткачество и прядение",
                          "Текстиль", "Рисование на воде Эбру", "Клеи и лак для полимерной глины", "Наперстки",
                          "Вязание", "Краски Холи", "Валяние", "Наборы для рисования песком", "Элементы интерьера",
                          "Свадебные товары", "Открытки", "Заготовки для декорирования", "Подсвечники и аромалампы",
                          "Наборы для создания фоторамок", "Декупаж", "Папье-маше", "Модели из дерева", "Кожевенное дело",
                          "Алмазные мозаики", "Лэмпворк", "Ароматы для дома", "Парфюмированный набор", "Карандаши",
                          "Цветной песок", "Значки", "Наборы для создания слепков", "Украшения на машину",
                          "Изготовление игрушек", "Пиньяты", "Пастель", "Глиттеры", "Флористика", "Досуг и творчество",
                          "Фотобутафория", "Картины и панно", "Аксессуары и материалы", "Куклы",  "Карточки",
                          "Наборы для гигиены", "Картины по номерам", "Воздушные шары", "Свечи и подсвечники",
                          "Антикварные газеты и журналы", "Пэчворк и квилтинг", "Раскраски", "Папертоли",
                          "Фотоаппараты мгновенной печати", "Билеты", "Наборы для создания причесок",
                          "Масса и тесто для лепки", "Наборы для создания духов", "Открытки и конверты",
                          "Цветы", "Наборы средств для лица", "Интерьерные миниатюры и сборка зданий",
                          "Косметические наборы", "Холсты, артборды и картон грунтованный", "Хранение коллекций",
                          "Аксессуары"],

    "технологии":        ["Компьютерные технологии", "Электроника", "Компьютеры",  "Периферия", "Игровые планшеты", 'Проекторы'],

    "мода":              ["Журналы и наклейки", "Мода", "Красота", "Шоппинг", "Этикетки и пробки", "Карнавальные товары",
                          "Подарочные наборы", "Марки", "Банкноты", "Журналы и газеты",
                          "Наборы для макияжа", "Испанский язык", "Английский язык", "Значки"],

    "красота":           ["Красота, здоровье и спорт", "Изготовление косметики", "Наборы косметики", "Косметические наборы",
                          "Наборы средств для лица", "Наборы для гигиены"],

    "шоппинг":           ["Всё для дома", "Подарочные наборы", "Сувениры и подарки"],

    "автомобили":        ["Автомобили и мотоциклы"],

    "рыбалка":           ["Инструменты и инвентарь", "Аксессуары"],

    "охота":             ["Инструменты и инвентарь", "Аксессуары"],

    "финансы":           ["Финансы", "Экономика", "Бухгалтерский учет и делопроизводство", "Инвестиции и недвижимость",
                          "Маркетинг и продажи", "Менеджмент и управление", "Банкноты", "Монеты"],

    "образование":       ["Бизнес-образование и развитие карьеры", "Развитие детей", "Учебная литература",
                          "Научно-популярная литература", "Детям и родителям", "Познавательная литература"],

    "здоровье":          ["Красота, здоровье и спорт", "Медицина", "Средства для ухода", "Наборы для гигиены"],

    "саморазвитие":      ["Саморазвитие", "Трансформационные психологические игры", "Психология"],

    # "домашние животные": ["Домашние животные", "Товары для животных"],

    "садоводство":       ["Домоводство и хобби", "Инструменты и инвентарь", "Садоводство", "Цветы"],

    "фотография":        ["Фотография", "Цифровые фоторамки", "Фотоаппараты мгновенной печати"],

    # "танцы":             ["Танцы"],
    "йога":              ["Красота, здоровье и спорт", "Йога"],

    # "волонтерство":      ["Волонтерство"],

    "рисование":         ["Рисование жидким акрилом (Флюид-арт)", "Мольберты и этюдники", "Штампы и трафареты",
                          "Скетчбуки", "Краски", "Уголь, сангина, сепия и соус", "Маркеры", "Наборы для росписи",
                          "Картриджи для 3D ручек", "Рисование на воде Эбру", "Наборы для рисования песком",
                          "Карандаши", "Пастель", "Раскраски"],

    "живопись":          ["Рисование жидким акрилом (Флюид-арт)", "Мольберты и этюдники", "Штампы и трафареты",
                          "Картины из пайеток", "Картины из эпоксидной смолы", "Средства для очищения кистей",
                          "Краски", "Уголь, сангина, сепия и соус", "Маркеры", "Наборы для росписи",  "Картриджи для 3D ручек",
                          "Рисование на воде Эбру", "Карандаши", "Пастель", "Картины и панно", "Картины по номерам"],

    "скульптура":        ["Наборы для лепки", "Глина для лепки", "Пластилин", "Гипс", "Масса и тесто для лепки"],

    "театр":             ["Театр", "Пьесы и драматургия"],

    "писательство":      ["Письменные принадлежности", "Бумага для черчения и рисования", "Карандаши"],

    "поэзия":            ["Поэзия"],

    "рукоделие":         ["Гобелены и стринг-арт", "Вышивание", "Изготовление косметики", "Наборы для лепки",
                          "Вышивание", "Штампы и трафареты", "Картины из пайеток", "Авторская посуда",
                          "Топперы для торта", "Выжигание и поделки из дерева", "Создание украшений",
                          "Пластилин", "Глина для лепки", "Принадлежности", "Оригами", "Реставрация и золочение",
                          "Шелкография", "Скетчбуки", "Краски", "Уголь, сангина, сепия и соус", "Маркеры",
                          "Наборы для росписи", "Подставки", "Картриджи для 3D ручек", "Шитье", "Поделки из бумаги",
                          "Веники для бань", "Аксессуары и атрибутика для гаданий", "Изготовление ножей",
                          "Ткачество и прядение", "Текстиль", "Рисование на воде Эбру", "Клеи и лак для полимерной глины",
                          "Наперстки", "Вязание", "Краски Холи", "Валяние", "Наборы для рисования песком",
                          "Элементы интерьера", "Свадебные товары", "Открытки", "Заготовки для декорирования",
                          "Подсвечники и аромалампы", "Наборы для создания фоторамок", "Декупаж", "Папье-маше",
                          "Модели из дерева", "Кожевенное дело", "Алмазные мозаики", "Лэмпворк", "Наборы для макияжа",
                          "Ароматы для дома", "Парфюмированный набор", "Карандаши", "Цветной песок", "Значки",
                          "Наборы для создания слепков", "Украшения на машину", "Изготовление игрушек", "Пиньяты",
                          "Пастель", "Глиттеры", "Флористика", "Досуг и творчество", "Фотобутафория", "Картины и панно",
                          "Аксессуары и материалы", "Куклы",  "Карточки", "Товары для рукоделия", "Наборы для гигиены",
                          "Картины по номерам", "Воздушные шары", "Свечи и подсвечники", "Антикварные газеты и журналы",
                          "Пэчворк и квилтинг", "Наборы косметики", "Раскраски", "Папертоли", "Наборы для создания причесок",
                          "Масса и тесто для лепки", "Наборы для создания духов", "Открытки и конверты", "Цветы",
                          "Наборы средств для лица", "Интерьерные миниатюры и сборка зданий", "Косметические наборы",
                          "Холсты, артборды и картон грунтованный", "Скрапбукинг", "Мыловарение", "Наборы для рисования",
                          "Аппликации", "Плетение"],

    "дизайн":            ["Дизайн", "Мольберты и этюдники", "Штампы и трафареты", "Бумага для черчения и рисования",
                          "Карандаши", "Наборы для создания фоторамок", "3D ручки"],

    "марки":             ["Марки", "Коллекционирование", "Хранение коллекций"],

    "монеты":            ["Монеты", "Коллекционирование", "Хранение коллекций"],

    "комиксы":           ["Комиксы", "Журналы и наклейки"],

    "антиквариат":       ["Антиквариат", "Минералы и окаменелости", "Амулеты, талисманы и ловцы снов",
                          "Античная литература", "Автографы и фото коллекционные", "Награды и кубки",
                          "Банкноты", "Монеты", "Антикварные газеты и журналы"],

    "игрушки":           ["Мягкие игрушки", "Игрушечное оружие", "Роботы и трансформеры", "Развивающие игры",
                          "Игрушки для малышей", "Конструкторы", "Игрушки-антистресс", "Сюжетно-ролевые игрушки",
                          "Модели и фигурки", "Игрушки для ванной", "Интерактивные и электронные игрушки", "Куклы",
                          "Военная техника и оружие", "Фигурки и аксессуары", "Игрушечный транспорт",
                          "Куклы и аксессуары", "Мыльные пузыри", "Игрушки для ванной", "Игрушечный транспорт",
                          "Игральные карты", "Головоломки", "Трансформационные психологические игры",
                          "Корабли и подводные лодки", "Мягкие игрушки", "Игрушечное оружие","Радиоуправляемые игрушки",
                          "3D ручки", "Кинетический песок", "Роботы и трансформеры", "Развивающие игры", "Игрушки для малышей",
                          "Конструкторы", "Игрушки-антистресс", "Пазлы",'Самолеты и космические корабли', "Копилки", "Слаймы"],
    "видеоигры":         ["Видеоигры", "Игры для Nintendo Switch", "Игры для Xbox Series", "Игры для Xbox 360",
                          "Игры для PlayStation Portable/Vita", "Игры для Xbox One", "Игры для всей семьи (Family)",
                          "Игры для Nintendo Wii/3DS", "Игры для PlayStation 4", "Игры для PlayStation 5",
                          "Игры для PlayStation 2/3", 'Очки виртуальной реальноcти для смартфонов'],
    "компьютерные игры": ["Компьютерные игры", "Игры для Nintendo Switch", "Игры для Xbox Series",
                          "Игры для Xbox 360", "Игры для PlayStation Portable/Vita", "Игры для Xbox One",
                          "Игры для всей семьи (Family)", "Игры для Nintendo Wii/3DS", "Игры для PlayStation 4",
                          "Игры для PlayStation 5", "Игры для PlayStation 2/3"],

    # "мобильные игры":    ["Мобильные игры"],

    "настольные игры":   ["Настольные игры", "Дополнения к настольным играм", "Ходилки", "Игральные карты",
                          "Головоломки", "Трансформационные психологические игры", "Корабли и подводные лодки",
                          "Дополнения", "Лото", "Карты Таро", "Руны",  "Сюжетно-ролевые игрушки", "Шахматы", "Шашки", "Нарды",
                          "Карточные игры"],

    "азартные игры":     ["Азартные игры", "Игральные карты", "Игры в рулетку", "Наборы для покера", "Карты оплаты", "Домино и маджонг"],
    "коллекционирование":['Банкноты', 'Наборы', 'Значки', 'Этикетки и пробки', 'Карточки',
                          'Куклы', 'Билеты', 'Автографы и фото коллекционные',
                          'Минералы и окаменелости', 'Открытки и конверты', 'Наперстки',
                          'Хранение коллекций', 'Журналы и наклейки', 'Монеты', 'Марки',
                          'Календари', 'Средства для ухода', 'Модели и фигурки', 'Насекомые', "Фигурки"]
}
# size = 18427
interests_codes = {}
for interest in interests_categories:
    codes = []
    for word in interests_categories[interest]:
        codes += data.loc[data['подподкатегория'] == word, 'код товара'].values.tolist()
    if codes:
        interests_codes[interest] = codes
        print(f'{interest} {interests_codes[interest]}')

with open('interests_categories.json', 'w', encoding='utf-8') as file:
    json.dump(interests_codes, file, indent=4,  ensure_ascii=False)

спорт [2130524, 482231739, 250970185, 250803970, 231648553, 31432890, 252734105, 1202755130, 658045563, 231648510, 1262906846, 31763354, 291992349, 324401964, 157804021, 1431518928, 1252526404, 388014458, 231648399, 211432418, 1358455694, 207921218, 267303955, 783514028, 819878142, 660370742, 158873698, 533841531, 250456382, 211432837, 249025521, 250876411, 1352666135, 1397875334, 820918597, 249179759, 355950751, 249166924, 250949612, 901137625, 250059248, 26419023, 34870845, 934325701, 892379875, 267320951, 249173671, 137652886, 250999058, 253327264, 181292157, 1413807149, 1171570200, 181292155, 24700170, 805198893, 27936698, 147055608, 871459123, 135462206, 730856796, 1282867657, 571118709, 166482309, 796387608, 256442249, 135892589, 805035664, 298278173, 269215469, 1427919963, 1275935926, 1418775173, 747554009, 229445055, 383685217, 587975281, 798108521, 1253970847, 855605187, 801644242, 1116580961, 166567508, 1469262973, 909776699, 229445173, 1112990878, 260533998, 981621799, 80164

In [13]:
from itertools import chain

codes = list(set(chain(*list(interests_codes.values()))))
encode_codes = {codes[i]: i for i in range(len(codes))}

interests_codes_encode = {}
for interest in interests_codes:
    codes = []
    for code in interests_codes[interest]:
        codes.append(encode_codes[code])
    if codes:
        interests_codes_encode[interest] = codes
        print(f'{interest} {interests_codes_encode[interest]}')

with open('interests_categories_encode.json', 'w', encoding='utf-8') as file:
    json.dump(interests_codes_encode, file, indent=4,  ensure_ascii=False)

спорт [333, 8951, 27, 15279, 5703, 4139, 13675, 2794, 15760, 5678, 14075, 5482, 14755, 15846, 13148, 8071, 1188, 4238, 5615, 6595, 12782, 4076, 7692, 15637, 11448, 15045, 7167, 9041, 5183, 6797, 10823, 2256, 1524, 12657, 7290, 5867, 12434, 15976, 6084, 8828, 3230, 3946, 2797, 5781, 4442, 16254, 2844, 13672, 14494, 15368, 9854, 15959, 7962, 9852, 13012, 11887, 9210, 12832, 13543, 16096, 16107, 245, 2587, 10519, 13529, 16380, 1811, 12187, 12017, 13268, 10869, 7767, 9594, 8829, 1736, 2311, 9584, 5457, 2471, 16414, 3886, 5585, 3814, 5598, 2909, 1796, 12951, 14214, 11883, 2839, 8443, 5307, 15099, 3887, 8901, 7336, 5461, 14954, 15265, 3180, 9994, 8520, 2176, 8867, 12608, 13874, 11130, 4876, 11568, 2476, 2426, 11578, 4646, 11041, 11877, 10973, 1440, 13474, 11147, 11666, 15252, 9246, 1823, 11997, 1408, 183, 16319, 6636, 4702, 15315, 12266, 10640]
музыка [4503, 14885, 8690, 3739, 2451, 12546, 2947, 5833, 13245, 361, 16325, 13934, 4991, 14770, 7538, 6439, 13025, 14047, 13688, 14313, 3696, 6836, 

In [14]:
#  пример данных
# user_data = {
#     'user_id': [1, 2, 3, 4],
#     'interests': [['спорт', 'технологии'], ['мода', 'еда'], ['технологии', 'спорт'], ['еда', 'мода']],
#     'products': [[1, 2], [3, 4], [2, 3], [1, 4]]
# }

In [47]:
user_vectors = []
user_id = 1

# Проходимся по всем интересам, чтобы каждый был использован
for interest in interests_categories.keys():

  # Находим все коды товаров для текущего интереса
  product_codes = interests_codes_encode[interest]

  # Создаем пользователей, пока не используем все коды товаров
  while product_codes:

    # Выбираем от 2 до 5 интересов (включая текущий)
    user_interests = random.sample(list(interests_categories.keys()), random.randint(2, 5))
    if interest not in user_interests:
      user_interests.append(interest)

    # Создаем список товаров для пользователя, выбирая от 1 до 5 товаров
    user_products = []
    num_products = random.randint(1, min(5, len(product_codes)))
    for _ in range(num_products):
      selected_product = random.choice(product_codes)
      user_products.append(selected_product)
      product_codes.remove(selected_product) # Удаляем использованный код


    user_vectors.append({
        'user_id': user_id,
        'interests': user_interests,
        'products': user_products
    })
    user_id += 1

# Если нужно больше 3000 анкет, добавляем пользователей с рандомными интересами и товарами
while len(user_vectors) < 3000:
  user_interests = random.sample(list(interests_categories.keys()), random.randint(2, 5))
  user_products = []
  for interest in user_interests:
    user_products.extend(random.sample(interests_codes_encode[interest], random.randint(1, 5)))
  user_vectors.append({
      'user_id': user_id,
      'interests': user_interests,
      'products': user_products
  })
  user_id += 1

print(user_vectors)
print(len(user_vectors))

ValueError: Sample larger than population or is negative

In [16]:
df_user_vectors = pd.DataFrame(user_vectors)
df_user_vectors.head()

,user_id,interests,products
0,1,"[охота, театр, здоровье, поэзия, спорт]","[5598, 2587, 13012]"
1,2,"[марки, автомобили, коллекционирование, рисова...","[14755, 245, 14494, 3946, 3887]"
2,3,"[технологии, настольные игры, скульптура, теат...","[4139, 4238, 9852]"
3,4,"[технологии, рыбалка, театр, спорт]",[11041]
4,5,"[путешествия, спорт]","[5307, 2476, 6797, 8867, 2256]"


#### пример

In [17]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Создание словаря для хранения векторов товаров
item_vectors = {}

# Функция для создания вектора товара
def create_item_vector(row):
    vector = []
    vector.append(row['оценка'])
    vector.append(row['количество отзывов'])
    vector.append(row['цена'])
    vector += [1 if cat == row['категория'] else 0 for cat in data['категория'].unique()]
    vector += [1 if subcat == row['подкатегория'] else 0 for subcat in data['подкатегория'].unique()]
    vector += [1 if subsubcat == row['подподкатегория'] else 0 for subsubcat in data['подподкатегория'].unique()]
    return vector

# Заполнение словаря векторами товаров
for index, row in data.iterrows():
    item_vectors[row['код товара']] = create_item_vector(row)

# Функция для получения рекомендаций
def get_recommendations(item_id, top_n=3):
    input_vector = item_vectors[item_id]
    similarities = {}
    for key, value in item_vectors.items():
        if key != item_id:
            similarities[key] = cosine_similarity([input_vector], [value])[0][0]
    return sorted(similarities, key=similarities.get, reverse=True)[:top_n]

# Пример использования
recommendations = get_recommendations(926943324)
print(f"Рекомендации для товара с кодом 926943324: {recommendations}")

# Вывод рекомендаций с информацией о товаре
data.loc[data['код товара'].isin(recommendations), ['название', 'описание', 'ссылка']]

Рекомендации для товара с кодом 926943324: [173139380, 161855043, 1036269459]


,название,описание,ссылка
3070,"Настольные классические игры ""Шахматы, шашки"" ...",,https://www.ozon.ru/product/nastolnye-klassich...
3812,"Подарочный набор DREAMBOX ""БРАТУ"" Подарок для ...",Условия хранения\n\nХранить в защищенном от пр...,https://www.ozon.ru/product/podarochnyy-nabor-...
14103,Декоративные Сухие Блестки для творчества и ру...,Комплектация\n\nДекоративные Сухие Блестки Lux...,https://www.ozon.ru/product/dekorativnye-suhie...


### **Предобработка текста:** лемматизация и стемминг для поля "текст", где "текст" = "название", "описание" и "ключевые слова" (объединение "категории", "подкатегории" и "подподкатегории"), чтобы привести слова к базовой форме и улучшить поиск.


In [ ]:
data['ключевые слова'] = data['категория'] + ', ' + data['подкатегория'] + ', ' + data['подкатегория']
data.drop(columns=['категория', 'подкатегория', 'подподкатегория'], axis=1, inplace=True)

In [ ]:
text_columns = [column for column in data.columns if data[column].dtype == 'object' and column != 'ссылка']
text_columns

['название', 'описание', 'ключевые слова']

In [ ]:
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]

    text = " ".join(tokens)

    return text

In [ ]:
data_ = data.copy(deep=True)

In [ ]:
data["текст"] = data[text_columns].apply(lambda x: ". ".join(x), axis=1)
data.head(2)

,название,описание,код товара,оценка,количество отзывов,цена,ссылка,ключевые слова,текст
0,Проектор для смартфона 2-го поколения своими р...,Проектор для смартфона (проектор для смартфона...,1320591182,3.3,13,808,https://www.ozon.ru/product/proektor-dlya-smar...,"Электроника, Телевизоры и видеотехника, Телеви...",Проектор для смартфона 2-го поколения своими р...
1,"Проектор 90411567, белый",3D увеличитель экрана телефона – это универсал...,619939085,4.4,205,300,https://www.ozon.ru/product/proektor-90411567-...,"Электроника, Телевизоры и видеотехника, Телеви...","Проектор 90411567, белый. 3D увеличитель экран..."


In [ ]:
data['текст'] = data['текст'].apply(preprocess_text)

In [ ]:
data[['текст']].head(3)

,текст
0,проектор смартфон 2 го поколение свой рука под...
1,проектор 90411567 белый 3d увеличитель экран т...
2,проектор 90411553 3d увеличительный экран ваш ...


### **Векторизация данных:** TF-IDF и word2vec для преобразования текстовых данных ("название", "описание" и "ключевые слова") в числовые векторы.


#### TD-IDF

In [ ]:
"""
# Создание TF-IDF векторайзера
vectorizer = TfidfVectorizer(max_features=10)  # Можно настроить max_features

# Обучение векторайзера и преобразование текста в векторы
tfidf_vectors = vectorizer.fit_transform(data["текст"])

# Преобразование в DataFrame
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names_out())

# Объединение с исходным DataFrame
data_tfidf = pd.concat([data, tfidf_df], axis=1)

data_tfidf.head(2)
"""

'\n# Создание TF-IDF векторайзера\nvectorizer = TfidfVectorizer(max_features=10)  # Можно настроить max_features\n\n# Обучение векторайзера и преобразование текста в векторы\ntfidf_vectors = vectorizer.fit_transform(data["текст"])\n\n# Преобразование в DataFrame\ntfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names_out())\n\n# Объединение с исходным DataFrame\ndata_tfidf = pd.concat([data, tfidf_df], axis=1)\n\ndata_tfidf.head(2)\n'

#### Word2Vec

In [ ]:
# # Подготовка текстов для Word2Vec
# text_corpus = [text.split() for text in data["текст"]]

# # Обучение Word2Vec модели
# model = Word2Vec(text_corpus, vector_size=10, window=5, min_count=5, workers=4)  # Настройте параметры

# # Получение векторов для каждого текста
# word2vec_vectors = []
# for text in text_corpus:
#     word_vectors = [model.wv[word] for word in text if word in model.wv]
#     if word_vectors:
#         text_vector = np.mean(word_vectors, axis=0)
#     else:
#         text_vector = np.zeros(model.vector_size)
#     word2vec_vectors.append(text_vector)

# # Преобразование в DataFrame
# word2vec_df = pd.DataFrame(word2vec_vectors)

# # Объединение с исходным DataFrame
# data_word2vec = pd.concat([data, word2vec_df], axis=1)

# data_word2vec.head(2)

### **Нормализация числовых данных:** Нормализовация данные о цене и количестве отзывов, чтобы они имели одинаковый масштаб.


In [ ]:
# scaler = MinMaxScaler().fit(data[normalize_column])

# data_normalize = scaler.transform(data[normalize_column])
# data[normalize_column] = pd.DataFrame(data_normalize, columns = normalize_column)

# data[normalize_column].head(2)

In [ ]:
# normalize_column = [column for column in data.columns if data[column].dtype != 'object' and column != 'код товара']
# data[normalize_column].head(2)

## **2.  Модель:**

* **Контент-базированная фильтрация:**
    * Вычислить сходство между векторами запроса пользователя и векторами товаров.
    * Для этого можно использовать косинусную меру или евклидово расстояние.
    * Рекомендуйте товары с наибольшим сходством.

Вычисление сходства между векторами запроса пользователя (кому, на какой праздник подарок), интересы и векторами товаров (название + описание + категории) с помощью  `linear_kernel`



In [ ]:
def filter_products(data, query, interests, price_range):
    # Пример фильтрации по ценовому диапазону
    filtered_data = data.loc[(data['цена'] >= price_range[0]) & (data['цена'] <= price_range[1])]

    # Пример фильтрации по интересам
    tfidf = TfidfVectorizer(stop_words=russian_stopwords)
    tfidf_matrix = tfidf.fit_transform(filtered_data['текст'])
    query = query + ' ' + ' '.join(interests)
    query_vec = tfidf.transform([query])

    filtered_data['similarity'] = linear_kernel(query_vec, tfidf_matrix).flatten()

    return filtered_data.sort_values(by=['similarity', 'оценка', 'количество отзывов'], ascending=False)[:10]

In [ ]:
# Пример запроса пользователя
query = 'подарок папе на день рождения 30 лет'
interests = ['машины', 'электроника']
price_range = (500, 2000)

# Получение рекомендаций
filter_products(data, query, interests, price_range)[['название', 'ключевые слова',  'цена', 'ссылка', 'код товара']]

,название,ключевые слова,цена,ссылка,код товара
235,Аккумуляторный блок,"Электроника, Аксессуары для электроники, Аксес...",905,https://www.ozon.ru/product/akkumulyatornyy-bl...,1505883105
271,Сделай сам Электронные шестеренки и механическ...,"Электроника, Аксессуары для электроники, Аксес...",1220,https://www.ozon.ru/product/sdelay-sam-elektro...,1391730221
254,Power Functions 8883/1 Средний мотор M-Motor с...,"Электроника, Аксессуары для электроники, Аксес...",1163,https://www.ozon.ru/product/power-functions-88...,1217816637
253,Дополнение к набору робототехники / Ресурсный ...,"Электроника, Аксессуары для электроники, Аксес...",1265,https://www.ozon.ru/product/dopolnenie-k-nabor...,646844950
265,"Шасси основа для мобильных роботов ардуино, ма...","Электроника, Аксессуары для электроники, Аксес...",1677,https://www.ozon.ru/product/shassi-osnova-dlya...,1142343926
236,WEDO2.0 Датчик наклона/45305/200+ чертежи,"Электроника, Аксессуары для электроники, Аксес...",782,https://www.ozon.ru/product/wedo2-0-datchik-na...,1313345590
233,WEDO2.0 Датчик перемещения/45304/200+ рисунков,"Электроника, Аксессуары для электроники, Аксес...",782,https://www.ozon.ru/product/wedo2-0-datchik-pe...,1313337750
239,Электродвигатель Электрический Средний линейны...,"Электроника, Аксессуары для электроники, Аксес...",861,https://www.ozon.ru/product/elektrodvigatel-el...,1479801517
3839,Подарочные наборы для женщин: подарок маме сес...,"Продукты питания, Выпечка и сладости, Выпечка ...",1439,https://www.ozon.ru/product/podarochnye-nabory...,1088337894
11008,"Линейка для квилтинга и пэчворка, 30 x 30 см, ...","Хобби и творчество, Рукоделие, Рукоделие",1119,https://www.ozon.ru/product/lineyka-dlya-kvilt...,1528410848


## **3.  Гибридный подход:**

* Комбинируйте контент-базированную и коллаборативную фильтрацию для большей точности рекомендаций.



## **4. Ранжирование и фильтрация:**

* **Ранжирование:** Отсортируйте рекомендации по убыванию сходства или прогнозируемого рейтинга.
* **Фильтрация:** Примените фильтры по цене, категории, подкатегории и т.д., чтобы сузить список рекомендаций в соответствии с предпочтениями пользователя.



## **5.  Дополнительные функции:**

* **Персонализация:** Используйте информацию о прошлых взаимодействиях пользователя с системой для повышения релевантности рекомендаций.
* **Разнообразие:** Стремитесь к разнообразию в рекомендациях, избегая предлагать слишком похожие товары.
* **Объяснение:** Отобразите пользователю, почему ему были рекомендованы те или иные товары.

## **Использование синтетических данных о пользователях и нейросети для обучения**

In [18]:
# user_df     -> df_user_vectors
# product_df  -> data

# Подготовка данных для обучения
interactions = []

for user_id, products in zip(df_user_vectors['user_id'], df_user_vectors['products']):
    for product in products:
        interactions.append((user_id, product, 1))  # Положительные взаимодействия

# Генерация отрицательных взаимодействий
all_products = set([i for i in range(16443)]) # set(data['код товара'])
for user_id, products in zip(df_user_vectors['user_id'], df_user_vectors['products']):
    negative_samples = list(all_products - set(products))
    negative_samples = np.random.choice(negative_samples, len(products), replace=False)
    for product in negative_samples:
        interactions.append((user_id, product, 0))  # Отрицательные взаимодействия

# Преобразование в DataFrame
interactions_df = pd.DataFrame(interactions, columns=['user_id', 'product_id', 'interaction'])
interactions_df.head()

,user_id,product_id,interaction
0,1,5598,1
1,1,2587,1
2,1,13012,1
3,2,14755,1
4,2,245,1


In [19]:
# создание модели

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# # Параметры
# num_users = df_user_vectors['user_id'].nunique()
# num_products = data['код товара'].nunique()
# embedding_dim = 50

# Проверка максимальных значений user_id и product_id
max_user_id = interactions_df['user_id'].max()
max_product_id = interactions_df['product_id'].max()

print(f'Max user_id: {max_user_id}, Max product_id: {max_product_id}')

# Определение количества пользователей и товаров
num_users = max_user_id + 1
num_products = max_product_id + 1
embedding_dim = 50

class NCF(nn.Module):
    def __init__(self, num_users, num_products, embedding_dim):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users + 1, embedding_dim)
        self.product_embedding = nn.Embedding(num_products + 1, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, user, product):
        user_emb = self.user_embedding(user)
        product_emb = self.product_embedding(product)
        x = torch.cat([user_emb, product_emb], dim=-1)
        x = self.dropout(torch.relu(self.fc1(x)))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

model = NCF(num_users, num_products, embedding_dim)

Max user_id: 10421, Max product_id: 16442


In [20]:
num_users, num_products

(10422, 16443)

In [21]:
#  подготовка данных для DataLoader и обучение модели

class InteractionDataset(Dataset):
    def __init__(self, interactions_df):
        self.user_ids = torch.tensor(interactions_df['user_id'].values, dtype=torch.long)
        self.product_ids = torch.tensor(interactions_df['product_id'].values, dtype=torch.long)
        self.labels = torch.tensor(interactions_df['interaction'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.product_ids[idx], self.labels[idx]

dataset = InteractionDataset(interactions_df)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [22]:
# Оптимизатор и функция потерь
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for user_ids, product_ids, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(user_ids, product_ids).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader)}')

Epoch 1/10, Loss: 0.694159709296764
Epoch 2/10, Loss: 0.6932771479230656
Epoch 3/10, Loss: 0.6929208471857268
Epoch 4/10, Loss: 0.6897711620095214
Epoch 5/10, Loss: 0.6811237522332373
Epoch 6/10, Loss: 0.667209775743568
Epoch 7/10, Loss: 0.6492029300867376
Epoch 8/10, Loss: 0.6254170041874519
Epoch 9/10, Loss: 0.5976508957052501
Epoch 10/10, Loss: 0.5663085282769206


In [31]:
data["code"] = data["код товара"].map(encode_codes)
# data["code"]  = data["code"].astype(np.int64)
data[['код товара', "code"]]

,код товара,code
0,1320591182,3945.0
1,619939085,686.0
2,709233698,1535.0
3,1234798720,1090.0
5,1431215806,3880.0
...,...,...
22584,1467973262,16265.0
22585,1526203839,753.0
22586,1353267350,7189.0
22588,1142878789,14647.0


In [33]:
data_ = data.copy(deep=True)
data_.dropna(inplace=True)
data_["code"]  = data_["code"].astype(np.int64)
data_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16443 entries, 0 to 22589
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   название            16443 non-null  object 
 1   описание            16443 non-null  object 
 2   код товара          16443 non-null  int64  
 3   оценка              16443 non-null  float64
 4   количество отзывов  16443 non-null  int64  
 5   цена                16443 non-null  int64  
 6   категория           16443 non-null  object 
 7   подкатегория        16443 non-null  object 
 8   подподкатегория     16443 non-null  object 
 9   ссылка              16443 non-null  object 
 10  code                16443 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 1.5+ MB


In [42]:
#  генерация рекомендаций
# user_df     -> df_user_vectors
# product_df  -> data

# Функция для получения рекомендаций
def get_user_recommendations(user_id, model, user_df, product_df, top_k=5):
    user_idx = user_df[user_df['user_id'] == user_id].index[0]
    all_products = torch.tensor(product_df['code'].values, dtype=torch.long)
    user_tensor = torch.tensor([user_idx] * len(all_products), dtype=torch.long)

    model.eval()
    with torch.no_grad():
        predictions = model(user_tensor, all_products).squeeze()

    top_indices = predictions.argsort(descending=True)[:top_k].cpu().numpy()
    recommended_products = product_df.iloc[top_indices]

    return recommended_products

# Пример получения рекомендаций для пользователя с id 1
recommendations = get_user_recommendations(100, model, df_user_vectors, data_)
recommendations

,название,описание,код товара,оценка,количество отзывов,цена,категория,подкатегория,подподкатегория,ссылка,code
21322,"Кукла коллекционная керамика ""Агата в бело-зел...",Наш товар высшего качества. Состав и комплектн...,1545116929,0.0,0,2043,Антиквариат и коллекционирование,Коллекционирование,Куклы,https://www.ozon.ru/product/kukla-kollektsionn...,3636
21886,Альбом Stray kids The sound фотобук Стрэй кидс...,Фотоальбом Stray Kids - это сборник ярких фото...,1416816528,4.9,12,555,Антиквариат и коллекционирование,Коллекционирование,Журналы и наклейки,https://www.ozon.ru/product/albom-stray-kids-t...,13342
13710,"Гамма Гуашь 16 шт., 40 мл./ 1360 г.",Комплектация\n\nСостав набора (каждого цвета п...,317226456,4.9,1624,1108,Хобби и творчество,Рисование,Краски,https://www.ozon.ru/product/gamma-guash-16-sht...,16172
17274,"3Д РУЧКА Spider Pen SMART, с набором пластика ...","Длина рулона, м\n\n10",250897675,4.8,1060,2172,Хобби и творчество,3D-ручки и аксессуары,3D ручки,https://www.ozon.ru/product/3d-ruchka-spider-p...,13014
15832,"Candy Clay / Набор для лепки ""Большой набор По...",Да-да! Это самый большой и полный набор для ле...,223930388,4.8,452,4693,Хобби и творчество,Лепка и скульптура,Глина для лепки,https://www.ozon.ru/product/candy-clay-nabor-d...,13404


In [44]:
df_user_vectors[df_user_vectors['user_id'] == 100]

,user_id,interests,products
99,100,"[саморазвитие, кулинария, кино, здоровье, музыка]","[12914, 13462, 13144, 12920]"


In [45]:
#  оценка качества

def precision_recall_at_k(user_id, model, user_df, product_df, k=5):
    user_idx = user_df[user_df['user_id'] == user_id].index[0]
    true_products = user_df[user_df['user_id'] == user_id]['products'].values[0]

    recommended_products = get_user_recommendations(user_id, model, user_df, product_df, top_k=k)['code'].values
    relevant = set(true_products)
    recommended = set(recommended_products)

    precision = len(relevant & recommended) / k
    recall = len(relevant & recommended) / len(relevant)

    return precision, recall

# Пример оценки для пользователя с id 1
precision, recall = precision_recall_at_k(17, model, df_user_vectors, data_)
print(f'Precision@5: {precision}')
print(f'Recall@5: {recall}')

Precision@5: 0.0
Recall@5: 0.0
